In [38]:
import pandas as pd
import json
import os
import deepsig
from IPython.display import display

In [39]:
cols = ['dataset', 'method', 'fitness_rule', 'fitness', 'ACC', 'MCC', 'f1_score', 'avg_odds_diff', 'stat_par_diff', 'eq_opp_diff']

In [40]:
mlp_baseline_results = pd.read_csv('simple_mlp_results.csv')
mlp_baseline_results.replace({'simple_mlp_initializer': r'MLP'}, inplace=True)

mlp_standard_l2_results = pd.read_csv('mlp_standard_l2_results.csv')
mlp_standard_l2_results.replace({'mlp_standard_l2_initializer': r'MLP+L2'}, inplace=True)

mlp_preg_results = pd.read_csv('mlp_preg_results.csv')
mlp_preg_results.replace({'mlp_preg_initializer': r'MLP+RPR$_{\rho}$'}, inplace=True)

mlp_sreg_results = pd.read_csv('mlp_sreg_results.csv')
mlp_sreg_results.replace({'mlp_sreg_initializer': r'MLP+RPR$_{\rho_s}$'}, inplace=True)

mlp_kreg_results = pd.read_csv('mlp_kreg_results.csv')
mlp_kreg_results.replace({'mlp_kreg_initializer': r'MLP+RPR$_{\tau}$'}, inplace=True)

mlp_xi_reg_results = pd.read_csv('mlp_xi_reg_results.csv')
mlp_xi_reg_results.replace({'mlp_xi_reg_initializer': r'MLP+RPR$_{\xi}$'}, inplace=True)

ftl_baseline_results = pd.read_csv('ftl_mlp_results.csv')
ftl_baseline_results.replace({'ftl_mlp_initializer': r'FTL'}, inplace=True)

ftl_xi_reg_results = pd.read_csv('ftl_mlp_xi_reg_results.csv')
ftl_xi_reg_results.replace({'ftl_mlp_xi_reg_initializer': r'FTL+RPR$_{\xi}$'}, inplace=True)


results = pd.concat([mlp_baseline_results, mlp_standard_l2_results, mlp_preg_results, mlp_sreg_results, mlp_kreg_results, mlp_xi_reg_results, ftl_baseline_results, ftl_xi_reg_results], ignore_index=True)


In [41]:
results.replace({'adult_dataset_reader': 'Adult Income', 'compas_dataset_reader': 'Compas Recidivism', 'german_dataset_reader': 'German Credit', 'bank_dataset_reader': 'Bank Marketing'}, inplace=True)
results.rename(columns={'avg_odds_diff': 'Equalized Odds', 'stat_par_diff': 'Statistical Parity', 'eq_opp_diff': 'Equal Opportunity', 'MCC': 'Mathew Correlation', 'ACC': 'Accuracy'}, inplace=True)

In [42]:
fitness_rules_target_metrics = {
    'mcc_parity': {'performance': 'Mathew Correlation', 'fairness': 'Statistical Parity'},
    'mcc_opportunity': {'performance': 'Mathew Correlation', 'fairness': 'Equal Opportunity'},
    'mcc_odds': {'performance': 'Mathew Correlation', 'fairness': 'Equalized Odds'},
    'acc_parity': {'performance': 'Accuracy', 'fairness': 'Statistical Parity'},
    'acc_opportunity': {'performance': 'Accuracy', 'fairness': 'Equal Opportunity'},
    'acc_odds': {'performance': 'Accuracy', 'fairness': 'Equalized Odds'}
}

fitness_rules_target_metrics = {
    'mcc_parity': ('Mathew Correlation', 'Statistical Parity'),
    'mcc_opportunity': ('Mathew Correlation', 'Equal Opportunity'),
    'mcc_odds': ('Mathew Correlation', 'Equalized Odds'),
    'acc_parity': ('Accuracy', 'Statistical Parity'),
    'acc_opportunity': ('Accuracy', 'Equal Opportunity'),
    'acc_odds': ('Accuracy', 'Equalized Odds')
}
fitness_rules_abvr = {
    'mcc_parity': 'Max(MCC - Stat. Parity)',
    'mcc_opportunity': 'Max(MCC - Eq. Odds)',
    'mcc_odds': 'Max(MCC - Eq. Opp.)',
    'acc_parity': 'Max(Acc - Stat. Parity)',
    'acc_opportunity': 'Max(Acc - Eq. Odds)',
    'acc_odds': 'Max(Acc - Eq. Opp.)'
}

results['Performance'] = 0
results['Fairness'] = 0
results['Fitness Rule'] = ''
for fitness_rule, (performance_metric, fairness_metric) in fitness_rules_target_metrics.items():
    results.loc[results.fitness_rule == fitness_rule,'Performance'] = results.loc[results.fitness_rule == fitness_rule,performance_metric]
    results.loc[results.fitness_rule == fitness_rule,'Fairness'] = results.loc[results.fitness_rule == fitness_rule,fairness_metric]
    results.loc[results.fitness_rule == fitness_rule,'Fitness Rule Abvr'] = fitness_rules_abvr[fitness_rule]
    results.loc[results.fitness_rule == fitness_rule,'Fitness Rule'] = 'Max(%s - %s)' % fitness_rules_target_metrics[fitness_rule]

/tmp/ipykernel_2226130/3340472993.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.58517004 0.58512823 0.5840247  0.5836371  0.57625737 0.5769683
 0.58075151 0.57811729 0.57147862 0.55847176 0.56402881 0.58191431
 0.58343253 0.57740222 0.51579307 0.30958237 0.37962947 0.55899189
 0.53558872 0.2598879  0.57717035 0.50086739 0.33275221 0.37929293
 0.47729109 0.29196869 0.29037704 0.52303609 0.27480633 0.38668781
 0.50626951 0.29960979 0.4042848  0.52119218 0.2936422  0.33522388
 0.49730721 0.25112662 0.35119067 0.510052   0.26843855 0.36081008
 0.52157495 0.30114722 0.42995147 0.48267704 0.29056691 0.23139881
 0.54223416 0.23653437 0.27041017 0.52121263 0.25759348 0.23912165
 0.5223892  0.27406652 0.22303564 0.54094974 0.27699558 0.30939251
 0.51731345 0.30339828 0.37825089 0.52113734 0.26323857 0.33963196
 0.5538225  0.27850228 0.30234984 0.51877126 0.29494101 0.2648939
 0.52742877 0.24492927 0.13407928 0.535

In [43]:
datasets = ['Adult Income', 'Bank Marketing', 'Compas Recidivism','German Credit']
datasets

['Adult Income', 'Bank Marketing', 'Compas Recidivism', 'German Credit']

In [44]:
fitness_rules = ['mcc_parity', 'mcc_opportunity', 'mcc_odds', 'acc_parity', 'acc_opportunity', 'acc_odds']
fitness_rules

['mcc_parity',
 'mcc_opportunity',
 'mcc_odds',
 'acc_parity',
 'acc_opportunity',
 'acc_odds']

In [45]:
methods = [r'MLP', r'MLP+L2', r'MLP+RPR$_{\rho}$', r'MLP+RPR$_{\rho_s}$', r'MLP+RPR$_{\tau}$', r'MLP+RPR$_{\xi}$', r'FTL', r'FTL+RPR$_{\xi}$']

In [46]:
grouped_results = results\
    .groupby(['fitness_rule', 'dataset', 'method'])\
    .agg({'fitness': ['mean', 'std', 'count'], 'Performance': ['mean', 'std'], 'Fairness': ['mean', 'std']})\
    #.sort_values(by=['fitness_rule', 'dataset', ('fitness','mean')], ascending=[False, True, False])
grouped_results['formatted_fitness'] = grouped_results.apply(lambda row: f"${row[('fitness', 'mean')]:.3f} (\pm{row[('fitness', 'std')]:.2f})$", axis=1)
grouped_results['formatted_performance'] = grouped_results.apply(lambda row: f"${row[('Performance', 'mean')]:.3f} (\pm{row[('Performance', 'std')]:.2f})$", axis=1)
grouped_results['formatted_fairness'] = grouped_results.apply(lambda row: f"${row[('Fairness', 'mean')]:.3f} (\pm{row[('Fairness', 'std')]:.2f})$", axis=1)
grouped_results

fitness                  \
                                                   mean       std count   
fitness_rule dataset       method                                         
acc_odds     Adult Income  FTL                 0.797546  0.022527    25   
                           FTL+RPR$_{\xi}$     0.805417  0.018968    15   
                           MLP                 0.760493  0.020149    15   
                           MLP+L2              0.762226  0.018550    15   
                           MLP+RPR$_{\rho_s}$  0.772452  0.020815    16   
...                                                 ...       ...   ...   
mcc_parity   German Credit MLP+L2              0.261057  0.102972    14   
                           MLP+RPR$_{\rho_s}$  0.192318  0.107623    16   
                           MLP+RPR$_{\rho}$    0.254748  0.071677    16   
                           MLP+RPR$_{\tau}$    0.255848  0.079738    14   
                           MLP+RPR$_{\xi}$     0.245571  0.070406    15   

                                              Performance            Fairness  \
                                                     mean       std      mean   
fitness_rule dataset       method                                               
acc_odds     Adult Income  FTL                   0.840876  0.007559  0.043330   
                           FTL+RPR$_{\xi}$       0.843169  0.005611  0.037752   
                           MLP                   0.849110  0.003824  0.088617   
                           MLP+L2                0.848631  0.002860  0.086405   
                           MLP+RPR$_{\rho_s}$    0.848646  0.002916  0.076194   
...                                                   ...       ...       ...   
mcc_parity   German Credit MLP+L2                0.373881  0.093515  0.112824   
                           MLP+RPR$_{\rho_s}$    0.291751  0.073053  0.099432   
                           MLP+RPR$_{\rho}$      0.341448  0.067708  0.086701   
                           MLP+RPR$_{\tau}$      0.341519  0.054298  0.085671   
                           MLP+RPR$_{\xi}$       0.329368  0.050637  0.083796   

                                                         formatted_fitness  \
                                                    std                      
fitness_rule dataset       method                                            
acc_odds     Adult Income  FTL                 0.023855  $0.798 (\pm0.02)$   
                           FTL+RPR$_{\xi}$     0.017930  $0.805 (\pm0.02)$   
                           MLP                 0.019533  $0.760 (\pm0.02)$   
                           MLP+L2              0.018986  $0.762 (\pm0.02)$   
                           MLP+RPR$_{\rho_s}$  0.020507  $0.772 (\pm0.02)$   
...                                                 ...                ...   
mcc_parity   German Credit MLP+L2              0.065766  $0.261 (\pm0.10)$   
                           MLP+RPR$_{\rho_s}$  0.060881  $0.192 (\pm0.11)$   
                           MLP+RPR$_{\rho}$    0.042336  $0.255 (\pm0.07)$   
                           MLP+RPR$_{\tau}$    0.056780  $0.256 (\pm0.08)$   
                           MLP+RPR$_{\xi}$     0.060963  $0.246 (\pm0.07)$   

                                              formatted_performance  \
                                                                      
fitness_rule dataset       method                                     
acc_odds     Adult Income  FTL                    $0.841 (\pm0.01)$   
                           FTL+RPR$_{\xi}$        $0.843 (\pm0.01)$   
                           MLP                    $0.849 (\pm0.00)$   
                           MLP+L2                 $0.849 (\pm0.00)$   
                           MLP+RPR$_{\rho_s}$     $0.849 (\pm0.00)$   
...                                                             ...   
mcc_parity   German Credit MLP+L2                 $0.374 (\pm0.09)$   
                           MLP+RPR$_{\rho_s}$     $0.292 (\

In [47]:
selected_columns = ['formatted_fitness', 'formatted_performance', 'formatted_fairness']
for fitness_rule in fitness_rules:
    grouped_results.loc[fitness_rule][selected_columns].to_latex(f'tables/grouped_results_{fitness_rule}_crp.tex')
     #.to_latex(f'tables/grouped_results_{fitness_rule}_crp.tex', columns=selected_columns))